# Prediction Using NN

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br></p>

In [2]:
def CreateDataFrame(types, level):
    data = pd.read_csv('../Extraction/'+types+"/"+level+'.csv')
    data["Class"] = level
    return data

In [3]:
def LoadData(types):
    frames = [CreateDataFrame(types,"Clean"), 
          CreateDataFrame(types,"Dirty (0-20)"), 
          CreateDataFrame(types,"Dirty (20-40)"), 
          CreateDataFrame(types,"Dirty (40-60)"),
          CreateDataFrame(types,"Dirty (60-80)"),
          CreateDataFrame(types,"Dirty (80-100)")]
    df = pd.concat(frames)
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

# Classification using NN

## Optimize Classfiers

In [18]:
def optimize(X_train, y_train):
    param_grid = [{'clf__hidden_layer_sizes': [(100,90,80,70,60,50,40,30,20,10,),
                                               (30,25,20,15,10,),
                                               (30,20,10,),(20,10,)], 'clf__alpha': [0.0001,0.001]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', MLPClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=5, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Cycle Work

In [19]:
def cycleWork(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    gridsearch = optimize(X_train, y_train)
    print(gridsearch.best_params_)
    estimator = gridsearch.best_estimator_
    
    y_train_pred = estimator.predict(X_train)
    trnscore = accuracy_score(y_train, y_train_pred)
    
    y_tst_pred = estimator.predict(X_test)
    tstscore = accuracy_score(y_test, y_tst_pred)
    cm = confusion_matrix(y_test, y_tst_pred)
    true_pos = np.diag(cm) 
    precision = np.sum(true_pos / np.sum(cm, axis=0))
    recall = np.sum(true_pos / np.sum(cm, axis=1))
    f1 = 2 * ((precision* recall)/(precision + recall))
    #f1None = f1_score(y_test, y_tst_pred, average=None)
    #f1micro = f1_score(y_test, y_tst_pred, average='micro')
    return [trnscore, tstscore, f1]

In [20]:
def AllWork(types,cycles = 5):
    data = []
    X,y = LoadData(types)
    for i in range(0,cycles):
        print(i)
        acc = cycleWork(X,y)
        print("Train Accuracy ",acc[0],", Test Accuracy",acc[1], "F1 ",acc[2])
        data.append([acc[0],acc[1],acc[2]])
        numpyarray = np.array(data)
        df = pd.DataFrame(numpyarray, columns = ["Train Acc", "Test Acc", "F1 Score"])
        #df.to_csv("NN-"+types+'.csv',index = False)

In [21]:
types = "Solar Data"
AllWork(types,1)
types = "NoBG"
AllWork(types,1)

0
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   49.3s finished


{'clf__alpha': 0.001, 'clf__hidden_layer_sizes': (30, 20, 10)}
Train Accuracy  0.980242478671 , Test Accuracy 0.97486535009 F1  5.85030183468
0
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   47.1s finished


{'clf__alpha': 0.0001, 'clf__hidden_layer_sizes': (30, 20, 10)}
Train Accuracy  0.980691513247 , Test Accuracy 0.971274685817 F1  5.81160296342
